# Modul Pengolahan Citra Digital (PCD)

```
pip install opencv-python scipy scikit-image matplotlib
```

## Daftar Isi

1. Definisi dan Manfaat PCD
2. Level-level dan Berbagai Operasi pada PCD
3. Konsep Persepsi Visual, Fenomena Visual, Spektrum Cahaya dan Gelombang Elektromagnetik
4. Akuisisi Citra, Proses Pembentukan Citra Digital, serta Penggunaan PCD
5. Operasi Titik Geometrik
6. Transformasi Tingkat Keabuan pada Citra
7. Interpretasi Histogram
8. Ekualisasi Histogram dan Histogram Matching
9. Operasi Morfologi
10. Konvolusi dan Filtering
11. Transformasi Domain Frekuensi
12. Pyramids, Edge Detection, dan Feature Detection/Description
13. Pengolahan Citra Berwarna
14. Segmentasi Citra
15. Kompresi Citra
16. Aplikasi PCD di Berbagai Bidang Sains dan Medical Imaging
17. State of the Art PCD di Berbagai Bidang


In [ ]:
# Import library yang diperlukan
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage, signal
from skimage import filters, feature, segmentation, color, exposure, morphology, transform
import warnings
warnings.filterwarnings('ignore')

# Set style untuk plotting
plt.style.use('default')
np.set_printoptions(precision=3, suppress=True)


# 1. Definisi dan Manfaat PCD

## Definisi Pengolahan Citra Digital (PCD)

**Pengolahan Citra Digital (Digital Image Processing)** adalah proses manipulasi dan analisis citra digital menggunakan algoritma komputer untuk meningkatkan kualitas citra, mengekstrak informasi, atau melakukan analisis otomatis.

### Karakteristik Citra Digital:
- Citra digital adalah representasi diskrit dari citra kontinyu
- Terdiri dari array dua dimensi (atau tiga dimensi untuk citra berwarna) dari nilai-nilai piksel
- Setiap piksel memiliki koordinat (x, y) dan nilai intensitas

## Manfaat PCD

1. **Peningkatan Kualitas Citra**: Memperbaiki kontras, mengurangi noise, memperjelas detail
2. **Ekstraksi Informasi**: Mendeteksi objek, mengukur parameter, menganalisis pola
3. **Otomasi**: Proses yang dapat dilakukan secara otomatis tanpa intervensi manusia
4. **Kompresi**: Mengurangi ukuran file citra untuk efisiensi penyimpanan dan transmisi
5. **Restorasi**: Memperbaiki citra yang rusak atau terdegradasi
6. **Analisis Medis**: Diagnosis melalui citra medis (X-ray, CT scan, MRI)
7. **Pengenalan Pola**: Face recognition, OCR, deteksi objek
8. **Computer Vision**: Sistem yang dapat "melihat" dan memahami citra


# 2. Level-level dan Berbagai Operasi pada PCD

## Level-level Pengolahan Citra

### 1. **Level Rendah (Low-level Processing)**
- Operasi pada level piksel
- Input dan output berupa citra
- Contoh: noise reduction, sharpening, contrast enhancement

### 2. **Level Menengah (Mid-level Processing)**
- Operasi pada level objek/region
- Input: citra, Output: atribut objek
- Contoh: segmentasi, deteksi edge, deteksi corner

### 3. **Level Tinggi (High-level Processing)**
- Operasi pada level semantik
- Input: atribut objek, Output: interpretasi/pengetahuan
- Contoh: object recognition, scene understanding, decision making

## Kategori Operasi PCD

1. **Operasi Titik (Point Operations)**: Operasi pada setiap piksel secara independen
2. **Operasi Spasial (Spatial Operations)**: Operasi yang melibatkan neighborhood piksel
3. **Operasi Geometrik (Geometric Operations)**: Transformasi geometri seperti rotasi, scaling
4. **Operasi Morfologi (Morphological Operations)**: Operasi berbasis struktur
5. **Operasi Domain Frekuensi (Frequency Domain Operations)**: Operasi dalam domain Fourier/Wavelet


# 3. Konsep Persepsi Visual, Fenomena Visual, Spektrum Cahaya dan Gelombang Elektromagnetik

## Persepsi Visual

Persepsi visual adalah proses bagaimana sistem visual manusia menginterpretasikan informasi dari cahaya yang masuk ke mata.

### Komponen Sistem Visual:
- **Mata**: Sensor yang menangkap cahaya
- **Retina**: Mengandung sel fotoreseptor (rods dan cones)
- **Optic Nerve**: Mengirimkan sinyal ke otak
- **Visual Cortex**: Memproses informasi visual

## Fenomena Visual

1. **Adaptasi**: Kemampuan mata menyesuaikan dengan tingkat kecerahan
2. **Kontras**: Perbedaan intensitas antara objek dan latar belakang
3. **Acuity**: Ketajaman visual
4. **Color Vision**: Persepsi warna melalui tiga jenis cones (trichromacy)

## Spektrum Cahaya dan Gelombang Elektromagnetik

### Spektrum Elektromagnetik:
- **Visible Light**: 400-700 nm (violet hingga red)
- **Infrared**: > 700 nm
- **Ultraviolet**: < 400 nm
- **X-ray, Gamma ray**: Untuk medical imaging

### Warna dalam Spektrum Visible:
- **Violet**: ~400-450 nm
- **Blue**: ~450-495 nm
- **Green**: ~495-570 nm
- **Yellow**: ~570-590 nm
- **Orange**: ~590-620 nm
- **Red**: ~620-700 nm


In [ ]:
# Visualisasi spektrum elektromagnetik
wavelengths = np.linspace(400, 700, 300)
colors = plt.cm.rainbow(np.linspace(0, 1, 300))

fig, ax = plt.subplots(figsize=(12, 2))
for i, (w, c) in enumerate(zip(wavelengths, colors)):
    ax.axvspan(w, w+1, color=c, alpha=0.8)

ax.set_xlim(400, 700)
ax.set_xlabel('Wavelength (nm)')
ax.set_yticks([])
ax.set_title('Visible Light Spectrum')
plt.tight_layout()
plt.show()


# 4. Akuisisi Citra, Proses Pembentukan Citra Digital, serta Penggunaan PCD

## Akuisisi Citra

Akuisisi citra adalah proses menangkap citra dari dunia nyata menjadi bentuk digital.

### Komponen Sistem Akuisisi:
1. **Sumber Cahaya**: Illuminasi objek
2. **Lensa**: Memfokuskan cahaya
3. **Sensor**: Menangkap cahaya (CCD/CMOS)
4. **ADC (Analog-to-Digital Converter)**: Mengkonversi sinyal analog ke digital
5. **Storage**: Menyimpan data digital

## Proses Pembentukan Citra Digital

### Tahapan:
1. **Sampling**: Mengambil sampel citra kontinyu pada interval diskrit
2. **Quantization**: Mengkonversi nilai kontinyu menjadi nilai diskrit (levels)
3. **Encoding**: Menyimpan sebagai array numerik

### Parameter Penting:
- **Spatial Resolution**: Jumlah piksel (misal: 1920x1080)
- **Intensity Resolution**: Jumlah level intensitas (misal: 256 levels untuk 8-bit)
- **Color Depth**: Jumlah bit per channel (8-bit, 16-bit, dll)


In [ ]:
# Demonstrasi pembacaan dan properti citra digital
img = cv2.imread("images/mandrill.jpg", cv2.IMREAD_COLOR)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

print("Shape citra berwarna:", img.shape)
print("Shape citra grayscale:", img_gray.shape)
print("Tipe data:", img.dtype)
print("Range nilai:", img.min(), "-", img.max())

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].imshow(img_rgb)
axes[0].set_title('Citra Berwarna (RGB)')
axes[0].axis('off')

axes[1].imshow(img_gray, cmap='gray')
axes[1].set_title('Citra Grayscale')
axes[1].axis('off')
plt.tight_layout()
plt.show()


# 5. Operasi Titik Geometrik

Operasi geometrik mengubah posisi atau orientasi piksel dalam citra.

## 1. Flipping (Pencerminan)

### Horizontal Flip (Flopping)
### Vertical Flip (Flipping)


In [ ]:
# Flipping operations
img = cv2.imread("images/mandrill.jpg", cv2.IMREAD_COLOR)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Horizontal flip (flop)
flip_horizontal = cv2.flip(img_rgb, 1)

# Vertical flip (flip)
flip_vertical = cv2.flip(img_rgb, 0)

# Both horizontal and vertical
flip_both = cv2.flip(img_rgb, -1)

fig, axes = plt.subplots(2, 2, figsize=(10, 10))
axes[0,0].imshow(img_rgb)
axes[0,0].set_title('Original')
axes[0,0].axis('off')

axes[0,1].imshow(flip_horizontal)
axes[0,1].set_title('Horizontal Flip (Flopping)')
axes[0,1].axis('off')

axes[1,0].imshow(flip_vertical)
axes[1,0].set_title('Vertical Flip (Flipping)')
axes[1,0].axis('off')

axes[1,1].imshow(flip_both)
axes[1,1].set_title('Both Directions')
axes[1,1].axis('off')
plt.tight_layout()
plt.show()


## 2. Rotating (Rotasi)


In [ ]:
# Rotation operations
h, w = img_rgb.shape[:2]
center = (w // 2, h // 2)

# Rotate 45 degrees
M_45 = cv2.getRotationMatrix2D(center, 45, 1.0)
rotated_45 = cv2.warpAffine(img_rgb, M_45, (w, h))

# Rotate 90 degrees
M_90 = cv2.getRotationMatrix2D(center, 90, 1.0)
rotated_90 = cv2.warpAffine(img_rgb, M_90, (w, h))

# Rotate -30 degrees
M_neg30 = cv2.getRotationMatrix2D(center, -30, 1.0)
rotated_neg30 = cv2.warpAffine(img_rgb, M_neg30, (w, h))

fig, axes = plt.subplots(2, 2, figsize=(10, 10))
axes[0,0].imshow(img_rgb)
axes[0,0].set_title('Original')
axes[0,0].axis('off')

axes[0,1].imshow(rotated_45)
axes[0,1].set_title('Rotated 45°')
axes[0,1].axis('off')

axes[1,0].imshow(rotated_90)
axes[1,0].set_title('Rotated 90°')
axes[1,0].axis('off')

axes[1,1].imshow(rotated_neg30)
axes[1,1].set_title('Rotated -30°')
axes[1,1].axis('off')
plt.tight_layout()
plt.show()


## 3. Cropping (Pemotongan)


In [ ]:
# Cropping operations
h, w = img_rgb.shape[:2]

# Crop center region
crop_center = img_rgb[h//4:3*h//4, w//4:3*w//4]

# Crop specific region
crop_specific = img_rgb[50:200, 100:250]

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(img_rgb)
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].imshow(crop_center)
axes[1].set_title('Cropped Center')
axes[1].axis('off')

axes[2].imshow(crop_specific)
axes[2].set_title('Cropped Specific Region')
axes[2].axis('off')
plt.tight_layout()
plt.show()


## 4. Downsampling dan Upsampling

**Downsampling**: Mengurangi resolusi citra (mengecilkan ukuran)
**Upsampling**: Meningkatkan resolusi citra (memperbesar ukuran)


In [ ]:
# Downsampling and Upsampling
h, w = img_rgb.shape[:2]

# Downsampling - reduce size by half
downsampled = cv2.resize(img_rgb, (w//2, h//2), interpolation=cv2.INTER_LINEAR)

# Downsampling - reduce to 1/4 size
downsampled_quarter = cv2.resize(img_rgb, (w//4, h//4), interpolation=cv2.INTER_LINEAR)

# Upsampling - double the size
upsampled = cv2.resize(img_rgb, (w*2, h*2), interpolation=cv2.INTER_LINEAR)

# Upsampling with different interpolation methods
upsampled_cubic = cv2.resize(img_rgb, (w*2, h*2), interpolation=cv2.INTER_CUBIC)
upsampled_lanczos = cv2.resize(img_rgb, (w*2, h*2), interpolation=cv2.INTER_LANCZOS4)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes[0,0].imshow(img_rgb)
axes[0,0].set_title(f'Original ({w}x{h})')
axes[0,0].axis('off')

axes[0,1].imshow(downsampled)
axes[0,1].set_title(f'Downsampled 1/2 ({w//2}x{h//2})')
axes[0,1].axis('off')

axes[0,2].imshow(downsampled_quarter)
axes[0,2].set_title(f'Downsampled 1/4 ({w//4}x{h//4})')
axes[0,2].axis('off')

axes[1,0].imshow(upsampled)
axes[1,0].set_title(f'Upsampled 2x Linear ({w*2}x{h*2})')
axes[1,0].axis('off')

axes[1,1].imshow(upsampled_cubic)
axes[1,1].set_title(f'Upsampled 2x Cubic')
axes[1,1].axis('off')

axes[1,2].imshow(upsampled_lanczos)
axes[1,2].set_title(f'Upsampled 2x Lanczos')
axes[1,2].axis('off')
plt.tight_layout()
plt.show()


# 6. Transformasi Tingkat Keabuan pada Citra

Transformasi tingkat keabuan mengubah nilai intensitas piksel berdasarkan fungsi transformasi.

## 1. Operasi Aritmetika


In [ ]:
# Arithmetic operations on grayscale image
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Addition (brightening)
brightened = cv2.add(img_gray, 50)

# Subtraction (darkening)
darkened = cv2.subtract(img_gray, 50)

# Multiplication (contrast enhancement)
multiplied = cv2.multiply(img_gray, 1.5).astype(np.uint8)

# Division (contrast reduction)
divided = cv2.divide(img_gray, 1.5).astype(np.uint8)

# Image addition (blending)
img2 = cv2.imread("images/lincoln.png", cv2.IMREAD_GRAYSCALE)
if img2 is not None:
    img2_resized = cv2.resize(img2, (img_gray.shape[1], img_gray.shape[0]))
    blended = cv2.addWeighted(img_gray, 0.7, img2_resized, 0.3, 0)
else:
    blended = img_gray

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes[0,0].imshow(img_gray, cmap='gray')
axes[0,0].set_title('Original')
axes[0,0].axis('off')

axes[0,1].imshow(brightened, cmap='gray')
axes[0,1].set_title('Brightened (+50)')
axes[0,1].axis('off')

axes[0,2].imshow(darkened, cmap='gray')
axes[0,2].set_title('Darkened (-50)')
axes[0,2].axis('off')

axes[1,0].imshow(multiplied, cmap='gray')
axes[1,0].set_title('Multiplied (×1.5)')
axes[1,0].axis('off')

axes[1,1].imshow(divided, cmap='gray')
axes[1,1].set_title('Divided (÷1.5)')
axes[1,1].axis('off')

axes[1,2].imshow(blended, cmap='gray')
axes[1,2].set_title('Blended')
axes[1,2].axis('off')
plt.tight_layout()
plt.show()


## 2. Thresholding

Thresholding mengkonversi citra grayscale menjadi citra biner (hitam-putih).


In [ ]:
# Thresholding operations
# Simple thresholding
_, thresh_binary = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY)
_, thresh_binary_inv = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY_INV)
_, thresh_trunc = cv2.threshold(img_gray, 127, 255, cv2.THRESH_TRUNC)
_, thresh_tozero = cv2.threshold(img_gray, 127, 255, cv2.THRESH_TOZERO)
_, thresh_tozero_inv = cv2.threshold(img_gray, 127, 255, cv2.THRESH_TOZERO_INV)

# Adaptive thresholding
thresh_adaptive_mean = cv2.adaptiveThreshold(img_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
                                             cv2.THRESH_BINARY, 11, 2)
thresh_adaptive_gauss = cv2.adaptiveThreshold(img_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                               cv2.THRESH_BINARY, 11, 2)

# Otsu's thresholding
_, thresh_otsu = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

fig, axes = plt.subplots(3, 3, figsize=(15, 15))
axes[0,0].imshow(img_gray, cmap='gray')
axes[0,0].set_title('Original')
axes[0,0].axis('off')

axes[0,1].imshow(thresh_binary, cmap='gray')
axes[0,1].set_title('Binary')
axes[0,1].axis('off')

axes[0,2].imshow(thresh_binary_inv, cmap='gray')
axes[0,2].set_title('Binary Inverted')
axes[0,2].axis('off')

axes[1,0].imshow(thresh_trunc, cmap='gray')
axes[1,0].set_title('Truncated')
axes[1,0].axis('off')

axes[1,1].imshow(thresh_tozero, cmap='gray')
axes[1,1].set_title('To Zero')
axes[1,1].axis('off')

axes[1,2].imshow(thresh_tozero_inv, cmap='gray')
axes[1,2].set_title('To Zero Inverted')
axes[1,2].axis('off')

axes[2,0].imshow(thresh_adaptive_mean, cmap='gray')
axes[2,0].set_title('Adaptive Mean')
axes[2,0].axis('off')

axes[2,1].imshow(thresh_adaptive_gauss, cmap='gray')
axes[2,1].set_title('Adaptive Gaussian')
axes[2,1].axis('off')

axes[2,2].imshow(thresh_otsu, cmap='gray')
axes[2,2].set_title("Otsu's Thresholding")
axes[2,2].axis('off')
plt.tight_layout()
plt.show()


## 3. Lookup Tables (LUT)

Lookup Tables adalah metode efisien untuk transformasi intensitas menggunakan tabel pemetaan.


In [ ]:
# Lookup Tables (LUT) examples
# Create LUT for negative transformation
lut_negative = np.array([255 - i for i in np.arange(256)], dtype=np.uint8)

# Create LUT for gamma correction (gamma = 0.5)
gamma = 0.5
lut_gamma = np.array([((i / 255.0) ** (1.0 / gamma)) * 255 for i in np.arange(256)], dtype=np.uint8)

# Create LUT for contrast stretching
def create_contrast_lut(r1, s1, r2, s2):
    lut = np.zeros(256, dtype=np.uint8)
    for i in range(256):
        if i <= r1:
            lut[i] = int((s1 / r1) * i)
        elif i <= r2:
            lut[i] = int(((s2 - s1) / (r2 - r1)) * (i - r1) + s1)
        else:
            lut[i] = int(((255 - s2) / (255 - r2)) * (i - r2) + s2)
    return lut

lut_contrast = create_contrast_lut(50, 30, 200, 220)

# Apply LUTs
img_negative = cv2.LUT(img_gray, lut_negative)
img_gamma = cv2.LUT(img_gray, lut_gamma)
img_contrast = cv2.LUT(img_gray, lut_contrast)

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes[0,0].imshow(img_gray, cmap='gray')
axes[0,0].set_title('Original')
axes[0,0].axis('off')

axes[0,1].imshow(img_negative, cmap='gray')
axes[0,1].set_title('Negative (LUT)')
axes[0,1].axis('off')

axes[1,0].imshow(img_gamma, cmap='gray')
axes[1,0].set_title('Gamma Correction (LUT, γ=0.5)')
axes[1,0].axis('off')

axes[1,1].imshow(img_contrast, cmap='gray')
axes[1,1].set_title('Contrast Stretching (LUT)')
axes[1,1].axis('off')
plt.tight_layout()
plt.show()

# Plot LUT functions
fig, ax = plt.subplots(figsize=(8, 6))
x = np.arange(256)
ax.plot(x, lut_negative, label='Negative', linewidth=2)
ax.plot(x, lut_gamma, label='Gamma (γ=0.5)', linewidth=2)
ax.plot(x, lut_contrast, label='Contrast Stretching', linewidth=2)
ax.plot(x, x, 'k--', label='Identity', linewidth=1)
ax.set_xlabel('Input Intensity')
ax.set_ylabel('Output Intensity')
ax.set_title('Lookup Table Functions')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


# 7. Interpretasi Histogram

Histogram citra menunjukkan distribusi frekuensi nilai intensitas piksel dalam citra.


In [ ]:
# Histogram interpretation
hist = cv2.calcHist([img_gray], [0], None, [256], [0, 256])

# Calculate statistics
mean_intensity = np.mean(img_gray)
std_intensity = np.std(img_gray)
median_intensity = np.median(img_gray)

fig, axes = plt.subplots(1, 2, figsize=(15, 5))
axes[0].imshow(img_gray, cmap='gray')
axes[0].set_title('Grayscale Image')
axes[0].axis('off')

axes[1].plot(hist, color='black')
axes[1].axvline(mean_intensity, color='r', linestyle='--', label=f'Mean: {mean_intensity:.1f}')
axes[1].axvline(median_intensity, color='g', linestyle='--', label=f'Median: {median_intensity:.1f}')
axes[1].set_xlabel('Pixel Intensity')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Histogram')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Mean Intensity: {mean_intensity:.2f}")
print(f"Standard Deviation: {std_intensity:.2f}")
print(f"Median Intensity: {median_intensity:.2f}")
print(f"Min Intensity: {img_gray.min()}")
print(f"Max Intensity: {img_gray.max()}")

# Interpretasi histogram
if std_intensity < 30:
    print("\nInterpretasi: Citra memiliki kontras rendah (histogram sempit)")
elif std_intensity > 60:
    print("\nInterpretasi: Citra memiliki kontras tinggi (histogram lebar)")
else:
    print("\nInterpretasi: Citra memiliki kontras sedang")

if mean_intensity < 85:
    print("Citra cenderung gelap")
elif mean_intensity > 170:
    print("Citra cenderung terang")
else:
    print("Citra memiliki kecerahan sedang")


# 8. Ekualisasi Histogram dan Histogram Matching

## Ekualisasi Histogram

Ekualisasi histogram adalah teknik untuk meningkatkan kontras citra dengan menyebarkan distribusi intensitas secara merata.


In [ ]:
# Histogram Equalization
equ = cv2.equalizeHist(img_gray)

# Calculate histograms
hist_original = cv2.calcHist([img_gray], [0], None, [256], [0, 256])
hist_equalized = cv2.calcHist([equ], [0], None, [256], [0, 256])

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes[0,0].imshow(img_gray, cmap='gray')
axes[0,0].set_title('Original Image')
axes[0,0].axis('off')

axes[0,1].imshow(equ, cmap='gray')
axes[0,1].set_title('Histogram Equalized')
axes[0,1].axis('off')

axes[1,0].plot(hist_original, color='black')
axes[1,0].set_xlabel('Pixel Intensity')
axes[1,0].set_ylabel('Frequency')
axes[1,0].set_title('Original Histogram')
axes[1,0].grid(True, alpha=0.3)

axes[1,1].plot(hist_equalized, color='black')
axes[1,1].set_xlabel('Pixel Intensity')
axes[1,1].set_ylabel('Frequency')
axes[1,1].set_title('Equalized Histogram')
axes[1,1].grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# CLAHE (Contrast Limited Adaptive Histogram Equalization)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
clahe_img = clahe.apply(img_gray)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
axes[0].imshow(img_gray, cmap='gray')
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].imshow(equ, cmap='gray')
axes[1].set_title('Global Histogram Equalization')
axes[1].axis('off')

axes[2].imshow(clahe_img, cmap='gray')
axes[2].set_title('CLAHE (Adaptive)')
axes[2].axis('off')
plt.tight_layout()
plt.show()


## Histogram Matching (Specification)

Histogram matching mengubah histogram citra agar sesuai dengan histogram target.


In [ ]:
# Histogram Matching
from skimage import exposure

# Load target image for histogram matching
target_img = cv2.imread("images/lincoln.png", cv2.IMREAD_GRAYSCALE)
if target_img is None:
    # Create a synthetic target with different histogram
    target_img = np.random.normal(128, 50, img_gray.shape).astype(np.uint8)
    target_img = np.clip(target_img, 0, 255)

# Resize target to match source size
if target_img.shape != img_gray.shape:
    target_img = cv2.resize(target_img, (img_gray.shape[1], img_gray.shape[0]))

# Perform histogram matching
matched = exposure.match_histograms(img_gray, target_img, channel_axis=None)

# Calculate histograms
hist_source = cv2.calcHist([img_gray], [0], None, [256], [0, 256])
hist_target = cv2.calcHist([target_img], [0], None, [256], [0, 256])
hist_matched = cv2.calcHist([matched.astype(np.uint8)], [0], None, [256], [0, 256])

fig, axes = plt.subplots(3, 2, figsize=(12, 15))
axes[0,0].imshow(img_gray, cmap='gray')
axes[0,0].set_title('Source Image')
axes[0,0].axis('off')

axes[0,1].imshow(target_img, cmap='gray')
axes[0,1].set_title('Target Image')
axes[0,1].axis('off')

axes[1,0].imshow(matched, cmap='gray')
axes[1,0].set_title('Matched Image')
axes[1,0].axis('off')

axes[1,1].plot(hist_source, color='blue', label='Source')
axes[1,1].plot(hist_target, color='red', label='Target', alpha=0.7)
axes[1,1].set_xlabel('Pixel Intensity')
axes[1,1].set_ylabel('Frequency')
axes[1,1].set_title('Source vs Target Histogram')
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3)

axes[2,0].plot(hist_matched, color='green', label='Matched')
axes[2,0].plot(hist_target, color='red', label='Target', alpha=0.7)
axes[2,0].set_xlabel('Pixel Intensity')
axes[2,0].set_ylabel('Frequency')
axes[2,0].set_title('Matched vs Target Histogram')
axes[2,0].legend()
axes[2,0].grid(True, alpha=0.3)

axes[2,1].axis('off')
plt.tight_layout()
plt.show()


# 9. Operasi Morfologi

Operasi morfologi adalah operasi berbasis struktur yang bekerja pada bentuk dan struktur objek dalam citra biner.

## 1. Dilation (Dilasi) dan Erosion (Erosi)


In [ ]:
# Morphological operations - Dilation and Erosion
# Create binary image for demonstration
_, binary_img = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY)

# Define structuring element
kernel = np.ones((5, 5), np.uint8)

# Erosion
erosion = cv2.erode(binary_img, kernel, iterations=1)

# Dilation
dilation = cv2.dilate(binary_img, kernel, iterations=1)

# Different kernel shapes
kernel_cross = cv2.getStructuringElement(cv2.MORPH_CROSS, (5, 5))
kernel_ellipse = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

erosion_cross = cv2.erode(binary_img, kernel_cross, iterations=1)
dilation_ellipse = cv2.dilate(binary_img, kernel_ellipse, iterations=1)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes[0,0].imshow(binary_img, cmap='gray')
axes[0,0].set_title('Original Binary')
axes[0,0].axis('off')

axes[0,1].imshow(erosion, cmap='gray')
axes[0,1].set_title('Erosion (Rectangular)')
axes[0,1].axis('off')

axes[0,2].imshow(dilation, cmap='gray')
axes[0,2].set_title('Dilation (Rectangular)')
axes[0,2].axis('off')

axes[1,0].imshow(binary_img, cmap='gray')
axes[1,0].set_title('Original Binary')
axes[1,0].axis('off')

axes[1,1].imshow(erosion_cross, cmap='gray')
axes[1,1].set_title('Erosion (Cross)')
axes[1,1].axis('off')

axes[1,2].imshow(dilation_ellipse, cmap='gray')
axes[1,2].set_title('Dilation (Ellipse)')
axes[1,2].axis('off')
plt.tight_layout()
plt.show()


## 2. Opening dan Closing

**Opening** = Erosion diikuti Dilation (menghilangkan noise kecil, memisahkan objek)
**Closing** = Dilation diikuti Erosion (menutup lubang kecil, menyambungkan objek)


In [ ]:
# Opening and Closing
kernel = np.ones((5, 5), np.uint8)

# Opening
opening = cv2.morphologyEx(binary_img, cv2.MORPH_OPEN, kernel)

# Closing
closing = cv2.morphologyEx(binary_img, cv2.MORPH_CLOSE, kernel)

# Multiple iterations
opening_2 = cv2.morphologyEx(binary_img, cv2.MORPH_OPEN, kernel, iterations=2)
closing_2 = cv2.morphologyEx(binary_img, cv2.MORPH_CLOSE, kernel, iterations=2)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes[0,0].imshow(binary_img, cmap='gray')
axes[0,0].set_title('Original Binary')
axes[0,0].axis('off')

axes[0,1].imshow(opening, cmap='gray')
axes[0,1].set_title('Opening (1 iteration)')
axes[0,1].axis('off')

axes[0,2].imshow(closing, cmap='gray')
axes[0,2].set_title('Closing (1 iteration)')
axes[0,2].axis('off')

axes[1,0].imshow(binary_img, cmap='gray')
axes[1,0].set_title('Original Binary')
axes[1,0].axis('off')

axes[1,1].imshow(opening_2, cmap='gray')
axes[1,1].set_title('Opening (2 iterations)')
axes[1,1].axis('off')

axes[1,2].imshow(closing_2, cmap='gray')
axes[1,2].set_title('Closing (2 iterations)')
axes[1,2].axis('off')
plt.tight_layout()
plt.show()


## 3. Hit-Miss Operator

Hit-Miss operator digunakan untuk deteksi pola spesifik dalam citra biner.


In [ ]:
# Hit-Miss Operator
# Create a simple pattern to detect
kernel_hitmiss = np.array([[-1, -1, -1],
                           [ 1,  1,  1],
                           [-1, -1, -1]], dtype=np.int8)

hitmiss = cv2.morphologyEx(binary_img, cv2.MORPH_HITMISS, kernel_hitmiss)

# Alternative: using separate hit and miss kernels
kernel_hit = np.array([[0, 0, 0],
                       [1, 1, 1],
                       [0, 0, 0]], dtype=np.uint8)

kernel_miss = np.array([[1, 1, 1],
                        [0, 0, 0],
                        [1, 1, 1]], dtype=np.uint8)

# Manual hit-miss
hit = cv2.morphologyEx(binary_img, cv2.MORPH_ERODE, kernel_hit)
miss = cv2.morphologyEx(255 - binary_img, cv2.MORPH_ERODE, kernel_miss)
hitmiss_manual = cv2.bitwise_and(hit, miss)

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes[0,0].imshow(binary_img, cmap='gray')
axes[0,0].set_title('Original Binary')
axes[0,0].axis('off')

axes[0,1].imshow(hitmiss, cmap='gray')
axes[0,1].set_title('Hit-Miss (OpenCV)')
axes[0,1].axis('off')

axes[1,0].imshow(hit, cmap='gray')
axes[1,0].set_title('Hit Component')
axes[1,0].axis('off')

axes[1,1].imshow(hitmiss_manual, cmap='gray')
axes[1,1].set_title('Hit-Miss (Manual)')
axes[1,1].axis('off')
plt.tight_layout()
plt.show()


# 10. Konvolusi dan Filtering

Konvolusi adalah operasi fundamental dalam pengolahan citra untuk filtering, edge detection, dan feature extraction.

## 1. Konvolusi dan Filtering Dasar


In [ ]:
# Convolution and Filtering
# Define various kernels
# Identity kernel
kernel_identity = np.array([[0, 0, 0],
                            [0, 1, 0],
                            [0, 0, 0]])

# Box filter (averaging)
kernel_box = np.ones((5, 5)) / 25

# Custom kernel
kernel_custom = np.array([[-1, -1, -1],
                          [-1,  8, -1],
                          [-1, -1, -1]])

# Apply convolution
identity_result = cv2.filter2D(img_gray, -1, kernel_identity)
box_result = cv2.filter2D(img_gray, -1, kernel_box)
custom_result = cv2.filter2D(img_gray, -1, kernel_custom)

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes[0,0].imshow(img_gray, cmap='gray')
axes[0,0].set_title('Original')
axes[0,0].axis('off')

axes[0,1].imshow(identity_result, cmap='gray')
axes[0,1].set_title('Identity Kernel')
axes[0,1].axis('off')

axes[1,0].imshow(box_result, cmap='gray')
axes[1,0].set_title('Box Filter (Averaging)')
axes[1,0].axis('off')

axes[1,1].imshow(custom_result, cmap='gray')
axes[1,1].set_title('Custom Kernel (Sharpening)')
axes[1,1].axis('off')
plt.tight_layout()
plt.show()


## 2. Smoothing dengan Gaussian Convolution

Gaussian filter adalah filter smoothing yang efektif untuk mengurangi noise sambil mempertahankan edge.


In [ ]:
# Gaussian Smoothing
# Different sigma values
gaussian_1 = cv2.GaussianBlur(img_gray, (5, 5), 1.0)
gaussian_2 = cv2.GaussianBlur(img_gray, (5, 5), 2.0)
gaussian_3 = cv2.GaussianBlur(img_gray, (9, 9), 3.0)

# Create Gaussian kernel manually
def create_gaussian_kernel(size, sigma):
    kernel = np.zeros((size, size))
    center = size // 2
    for i in range(size):
        for j in range(size):
            x, y = i - center, j - center
            kernel[i, j] = np.exp(-(x**2 + y**2) / (2 * sigma**2))
    kernel = kernel / np.sum(kernel)
    return kernel

gaussian_kernel = create_gaussian_kernel(5, 1.0)
gaussian_manual = cv2.filter2D(img_gray, -1, gaussian_kernel)

# Integral Image (for fast box filtering)
# Note: OpenCV doesn't have direct integral image filtering, but we can demonstrate the concept
integral_img = cv2.integral(img_gray)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes[0,0].imshow(img_gray, cmap='gray')
axes[0,0].set_title('Original')
axes[0,0].axis('off')

axes[0,1].imshow(gaussian_1, cmap='gray')
axes[0,1].set_title('Gaussian (σ=1.0)')
axes[0,1].axis('off')

axes[0,2].imshow(gaussian_2, cmap='gray')
axes[0,2].set_title('Gaussian (σ=2.0)')
axes[0,2].axis('off')

axes[1,0].imshow(gaussian_3, cmap='gray')
axes[1,0].set_title('Gaussian (σ=3.0, 9x9)')
axes[1,0].axis('off')

axes[1,1].imshow(gaussian_manual, cmap='gray')
axes[1,1].set_title('Gaussian (Manual Kernel)')
axes[1,1].axis('off')

axes[1,2].imshow(integral_img, cmap='gray')
axes[1,2].set_title('Integral Image')
axes[1,2].axis('off')
plt.tight_layout()
plt.show()

# Visualize Gaussian kernel
plt.figure(figsize=(8, 6))
plt.imshow(gaussian_kernel, cmap='hot', interpolation='nearest')
plt.colorbar()
plt.title('Gaussian Kernel (5x5, σ=1.0)')
plt.axis('off')
plt.tight_layout()
plt.show()


## 3. Turunan Pertama: Image Gradient

Gradient citra menunjukkan perubahan intensitas, berguna untuk edge detection.


In [ ]:
# First Derivative: Image Gradient
# Sobel operators
sobel_x = cv2.Sobel(img_gray, cv2.CV_64F, 1, 0, ksize=3)
sobel_y = cv2.Sobel(img_gray, cv2.CV_64F, 0, 1, ksize=3)

# Convert to absolute values and scale
sobel_x = np.absolute(sobel_x)
sobel_y = np.absolute(sobel_y)
sobel_x = np.uint8(255 * sobel_x / np.max(sobel_x))
sobel_y = np.uint8(255 * sobel_y / np.max(sobel_y))

# Gradient magnitude
sobel_magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
sobel_magnitude = np.uint8(255 * sobel_magnitude / np.max(sobel_magnitude))

# Gradient direction
sobel_direction = np.arctan2(sobel_y.astype(float), sobel_x.astype(float))

# Prewitt operators
kernel_prewitt_x = np.array([[-1, 0, 1],
                              [-1, 0, 1],
                              [-1, 0, 1]])
kernel_prewitt_y = np.array([[-1, -1, -1],
                              [ 0,  0,  0],
                              [ 1,  1,  1]])

prewitt_x = cv2.filter2D(img_gray, cv2.CV_64F, kernel_prewitt_x)
prewitt_y = cv2.filter2D(img_gray, cv2.CV_64F, kernel_prewitt_y)
prewitt_x = np.absolute(prewitt_x)
prewitt_y = np.absolute(prewitt_y)
prewitt_x = np.uint8(255 * prewitt_x / np.max(prewitt_x))
prewitt_y = np.uint8(255 * prewitt_y / np.max(prewitt_y))
prewitt_magnitude = np.sqrt(prewitt_x**2 + prewitt_y**2)
prewitt_magnitude = np.uint8(255 * prewitt_magnitude / np.max(prewitt_magnitude))

fig, axes = plt.subplots(3, 3, figsize=(15, 15))
axes[0,0].imshow(img_gray, cmap='gray')
axes[0,0].set_title('Original')
axes[0,0].axis('off')

axes[0,1].imshow(sobel_x, cmap='gray')
axes[0,1].set_title('Sobel X')
axes[0,1].axis('off')

axes[0,2].imshow(sobel_y, cmap='gray')
axes[0,2].set_title('Sobel Y')
axes[0,2].axis('off')

axes[1,0].imshow(sobel_magnitude, cmap='gray')
axes[1,0].set_title('Sobel Magnitude')
axes[1,0].axis('off')

axes[1,1].imshow(sobel_direction, cmap='hsv')
axes[1,1].set_title('Sobel Direction')
axes[1,1].axis('off')

axes[1,2].imshow(prewitt_x, cmap='gray')
axes[1,2].set_title('Prewitt X')
axes[1,2].axis('off')

axes[2,0].imshow(prewitt_y, cmap='gray')
axes[2,0].set_title('Prewitt Y')
axes[2,0].axis('off')

axes[2,1].imshow(prewitt_magnitude, cmap='gray')
axes[2,1].set_title('Prewitt Magnitude')
axes[2,1].axis('off')

axes[2,2].axis('off')
plt.tight_layout()
plt.show()


## 4. Turunan Kedua: Laplacian of Gaussian (LoG) dan Difference of Gaussian (DoG)

Turunan kedua digunakan untuk deteksi zero-crossing pada edge.


In [ ]:
# Second Derivative: Laplacian of Gaussian (LoG) and Difference of Gaussian (DoG)
# Laplacian
laplacian = cv2.Laplacian(img_gray, cv2.CV_64F)
laplacian = np.absolute(laplacian)
laplacian = np.uint8(255 * laplacian / np.max(laplacian))

# LoG (Laplacian of Gaussian)
# Method 1: Gaussian blur then Laplacian
gaussian_for_log = cv2.GaussianBlur(img_gray, (5, 5), 1.0)
log_1 = cv2.Laplacian(gaussian_for_log, cv2.CV_64F)
log_1 = np.absolute(log_1)
log_1 = np.uint8(255 * log_1 / np.max(log_1))

# Method 2: Using LoG kernel directly
def create_log_kernel(size, sigma):
    kernel = np.zeros((size, size))
    center = size // 2
    for i in range(size):
        for j in range(size):
            x, y = i - center, j - center
            r_squared = x**2 + y**2
            kernel[i, j] = ((r_squared - 2 * sigma**2) / (sigma**4)) * \
                          np.exp(-r_squared / (2 * sigma**2))
    return kernel

log_kernel = create_log_kernel(9, 1.5)
log_2 = cv2.filter2D(img_gray, cv2.CV_64F, log_kernel)
log_2 = np.absolute(log_2)
log_2 = np.uint8(255 * log_2 / np.max(log_2))

# DoG (Difference of Gaussian)
gaussian_small = cv2.GaussianBlur(img_gray, (5, 5), 1.0)
gaussian_large = cv2.GaussianBlur(img_gray, (5, 5), 2.0)
dog = gaussian_small.astype(float) - gaussian_large.astype(float)
dog = np.absolute(dog)
dog = np.uint8(255 * dog / np.max(dog))

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes[0,0].imshow(img_gray, cmap='gray')
axes[0,0].set_title('Original')
axes[0,0].axis('off')

axes[0,1].imshow(laplacian, cmap='gray')
axes[0,1].set_title('Laplacian')
axes[0,1].axis('off')

axes[0,2].imshow(log_1, cmap='gray')
axes[0,2].set_title('LoG (Gaussian + Laplacian)')
axes[0,2].axis('off')

axes[1,0].imshow(log_2, cmap='gray')
axes[1,0].set_title('LoG (Direct Kernel)')
axes[1,0].axis('off')

axes[1,1].imshow(dog, cmap='gray')
axes[1,1].set_title('DoG (Difference of Gaussian)')
axes[1,1].axis('off')

axes[1,2].imshow(log_kernel, cmap='hot', interpolation='nearest')
axes[1,2].set_title('LoG Kernel')
axes[1,2].axis('off')
plt.tight_layout()
plt.show()


## 5. Nonlinear Filters

Filter nonlinier lebih efektif untuk menghilangkan noise sambil mempertahankan edge dibanding filter linier.

### Median Filter
### Mean-shift Filter
### Bilateral Filter


In [ ]:
# Nonlinear Filters
# Add some noise for demonstration
noise = np.random.normal(0, 25, img_gray.shape).astype(np.int16)
img_noisy = np.clip(img_gray.astype(np.int16) + noise, 0, 255).astype(np.uint8)

# Median Filter
median_3 = cv2.medianBlur(img_noisy, 3)
median_5 = cv2.medianBlur(img_noisy, 5)

# Bilateral Filter (edge-preserving smoothing)
bilateral = cv2.bilateralFilter(img_noisy, 9, 75, 75)

# Mean-shift Filter (using scipy)
try:
    from scipy.ndimage import uniform_filter
    mean_shift_approx = uniform_filter(img_noisy, size=5)
except:
    mean_shift_approx = cv2.blur(img_noisy, (5, 5))

# Comparison with Gaussian (linear filter)
gaussian_comparison = cv2.GaussianBlur(img_noisy, (5, 5), 1.5)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes[0,0].imshow(img_gray, cmap='gray')
axes[0,0].set_title('Original (No Noise)')
axes[0,0].axis('off')

axes[0,1].imshow(img_noisy, cmap='gray')
axes[0,1].set_title('Noisy Image')
axes[0,1].axis('off')

axes[0,2].imshow(median_5, cmap='gray')
axes[0,2].set_title('Median Filter (5x5)')
axes[0,2].axis('off')

axes[1,0].imshow(bilateral, cmap='gray')
axes[1,0].set_title('Bilateral Filter')
axes[1,0].axis('off')

axes[1,1].imshow(gaussian_comparison, cmap='gray')
axes[1,1].set_title('Gaussian Filter (Linear)')
axes[1,1].axis('off')

axes[1,2].imshow(mean_shift_approx, cmap='gray')
axes[1,2].set_title('Mean-shift Approximation')
axes[1,2].axis('off')
plt.tight_layout()
plt.show()

# Edge preservation comparison
edge_original = cv2.Canny(img_gray, 50, 150)
edge_gaussian = cv2.Canny(gaussian_comparison, 50, 150)
edge_bilateral = cv2.Canny(bilateral, 50, 150)
edge_median = cv2.Canny(median_5, 50, 150)

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes[0,0].imshow(edge_original, cmap='gray')
axes[0,0].set_title('Edges: Original')
axes[0,0].axis('off')

axes[0,1].imshow(edge_gaussian, cmap='gray')
axes[0,1].set_title('Edges: Gaussian (blurred)')
axes[0,1].axis('off')

axes[1,0].imshow(edge_bilateral, cmap='gray')
axes[1,0].set_title('Edges: Bilateral (preserved)')
axes[1,0].axis('off')

axes[1,1].imshow(edge_median, cmap='gray')
axes[1,1].set_title('Edges: Median (preserved)')
axes[1,1].axis('off')
plt.tight_layout()
plt.show()


# 11. Transformasi Domain Frekuensi

Transformasi domain frekuensi memungkinkan analisis dan manipulasi citra dalam domain frekuensi.

## 1. Fourier Transform

Fourier Transform mengkonversi citra dari domain spasial ke domain frekuensi.


In [ ]:
# Fourier Transform
from scipy.fft import fft2, fftshift, ifft2, ifftshift

# Compute 2D FFT
f_transform = fft2(img_gray)
f_shift = fftshift(f_transform)

# Magnitude spectrum
magnitude_spectrum = np.log(np.abs(f_shift) + 1)

# Phase spectrum
phase_spectrum = np.angle(f_shift)

# Reconstruct from magnitude only
magnitude_only = np.abs(f_shift)
reconstructed_magnitude = np.abs(ifft2(ifftshift(magnitude_only)))

# Reconstruct from phase only
phase_only = np.exp(1j * phase_spectrum)
reconstructed_phase = np.abs(ifft2(ifftshift(phase_only)))

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes[0,0].imshow(img_gray, cmap='gray')
axes[0,0].set_title('Original Image')
axes[0,0].axis('off')

axes[0,1].imshow(magnitude_spectrum, cmap='gray')
axes[0,1].set_title('Magnitude Spectrum (log scale)')
axes[0,1].axis('off')

axes[0,2].imshow(phase_spectrum, cmap='gray')
axes[0,2].set_title('Phase Spectrum')
axes[0,2].axis('off')

axes[1,0].imshow(img_gray, cmap='gray')
axes[1,0].set_title('Original')
axes[1,0].axis('off')

axes[1,1].imshow(reconstructed_magnitude, cmap='gray')
axes[1,1].set_title('Reconstructed (Magnitude Only)')
axes[1,1].axis('off')

axes[1,2].imshow(reconstructed_phase, cmap='gray')
axes[1,2].set_title('Reconstructed (Phase Only)')
axes[1,2].axis('off')
plt.tight_layout()
plt.show()


## 2. Discrete Fourier Transform (DFT)

DFT adalah implementasi diskrit dari Fourier Transform untuk citra digital.


In [ ]:
# Discrete Fourier Transform using OpenCV
dft = cv2.dft(np.float32(img_gray), flags=cv2.DFT_COMPLEX_OUTPUT)
dft_shift = np.fft.fftshift(dft)

magnitude_spectrum_cv = 20 * np.log(cv2.magnitude(dft_shift[:,:,0], dft_shift[:,:,1]) + 1)

# Inverse DFT
dft_ishift = np.fft.ifftshift(dft_shift)
img_back = cv2.idft(dft_ishift)
img_back = cv2.magnitude(img_back[:,:,0], img_back[:,:,1])

# Compare original and reconstructed
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(img_gray, cmap='gray')
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].imshow(magnitude_spectrum_cv, cmap='gray')
axes[1].set_title('DFT Magnitude Spectrum (OpenCV)')
axes[1].axis('off')

axes[2].imshow(img_back, cmap='gray')
axes[2].set_title('Reconstructed (Inverse DFT)')
axes[2].axis('off')
plt.tight_layout()
plt.show()

print(f"Reconstruction error: {np.mean(np.abs(img_gray.astype(float) - img_back)):.2f}")


## 3. Frequency Domain Filtering

Filtering dalam domain frekuensi: Highpass, Lowpass, Bandpass


In [ ]:
# Frequency Domain Filtering
rows, cols = img_gray.shape
crow, ccol = rows // 2, cols // 2

# Create masks
def create_lowpass_mask(rows, cols, radius):
    mask = np.zeros((rows, cols), np.uint8)
    center = (rows//2, cols//2)
    y, x = np.ogrid[:rows, :cols]
    mask_area = (x - center[1])**2 + (y - center[0])**2 <= radius**2
    mask[mask_area] = 1
    return mask

def create_highpass_mask(rows, cols, radius):
    return 1 - create_lowpass_mask(rows, cols, radius)

def create_bandpass_mask(rows, cols, inner_radius, outer_radius):
    mask = np.zeros((rows, cols), np.uint8)
    center = (rows//2, cols//2)
    y, x = np.ogrid[:rows, :cols]
    mask_area = ((x - center[1])**2 + (y - center[0])**2 >= inner_radius**2) & \
                ((x - center[1])**2 + (y - center[0])**2 <= outer_radius**2)
    mask[mask_area] = 1
    return mask

# Apply filters
f_transform = fft2(img_gray)
f_shift = fftshift(f_transform)

# Lowpass filter
lowpass_mask = create_lowpass_mask(rows, cols, 50)
f_shift_low = f_shift * lowpass_mask
f_ishift_low = ifftshift(f_shift_low)
img_lowpass = np.abs(ifft2(f_ishift_low))

# Highpass filter
highpass_mask = create_highpass_mask(rows, cols, 30)
f_shift_high = f_shift * highpass_mask
f_ishift_high = ifftshift(f_shift_high)
img_highpass = np.abs(ifft2(f_ishift_high))

# Bandpass filter
bandpass_mask = create_bandpass_mask(rows, cols, 20, 60)
f_shift_band = f_shift * bandpass_mask
f_ishift_band = ifftshift(f_shift_band)
img_bandpass = np.abs(ifft2(f_ishift_band))

fig, axes = plt.subplots(3, 3, figsize=(15, 15))
axes[0,0].imshow(img_gray, cmap='gray')
axes[0,0].set_title('Original')
axes[0,0].axis('off')

axes[0,1].imshow(lowpass_mask, cmap='gray')
axes[0,1].set_title('Lowpass Mask')
axes[0,1].axis('off')

axes[0,2].imshow(img_lowpass, cmap='gray')
axes[0,2].set_title('Lowpass Filtered')
axes[0,2].axis('off')

axes[1,0].imshow(img_gray, cmap='gray')
axes[1,0].set_title('Original')
axes[1,0].axis('off')

axes[1,1].imshow(highpass_mask, cmap='gray')
axes[1,1].set_title('Highpass Mask')
axes[1,1].axis('off')

axes[1,2].imshow(img_highpass, cmap='gray')
axes[1,2].set_title('Highpass Filtered')
axes[1,2].axis('off')

axes[2,0].imshow(img_gray, cmap='gray')
axes[2,0].set_title('Original')
axes[2,0].axis('off')

axes[2,1].imshow(bandpass_mask, cmap='gray')
axes[2,1].set_title('Bandpass Mask')
axes[2,1].axis('off')

axes[2,2].imshow(img_bandpass, cmap='gray')
axes[2,2].set_title('Bandpass Filtered')
axes[2,2].axis('off')
plt.tight_layout()
plt.show()


## 4. Discrete Wavelet Transform (DWT)

Wavelet Transform memberikan representasi multi-resolution dari citra.


In [ ]:
# Discrete Wavelet Transform
try:
    import pywt
    
    # Perform 2D DWT
    coeffs = pywt.dwt2(img_gray, 'haar')
    cA, (cH, cV, cD) = coeffs
    
    # Visualize coefficients
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    axes[0,0].imshow(cA, cmap='gray')
    axes[0,0].set_title('Approximation (cA)')
    axes[0,0].axis('off')
    
    axes[0,1].imshow(cH, cmap='gray')
    axes[0,1].set_title('Horizontal Detail (cH)')
    axes[0,1].axis('off')
    
    axes[1,0].imshow(cV, cmap='gray')
    axes[1,0].set_title('Vertical Detail (cV)')
    axes[1,0].axis('off')
    
    axes[1,1].imshow(cD, cmap='gray')
    axes[1,1].set_title('Diagonal Detail (cD)')
    axes[1,1].axis('off')
    plt.tight_layout()
    plt.show()
    
    # Reconstruct image
    img_reconstructed = pywt.idwt2(coeffs, 'haar')
    
    # Multi-level decomposition
    coeffs2 = pywt.wavedec2(img_gray, 'haar', level=2)
    cA2 = coeffs2[0]
    details2 = coeffs2[1:]
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    axes[0].imshow(img_gray, cmap='gray')
    axes[0].set_title('Original')
    axes[0].axis('off')
    
    axes[1].imshow(img_reconstructed, cmap='gray')
    axes[1].set_title('Reconstructed from DWT')
    axes[1].axis('off')
    plt.tight_layout()
    plt.show()
    
    print(f"Reconstruction error: {np.mean(np.abs(img_gray.astype(float) - img_reconstructed)):.2f}")
    
except ImportError:
    print("PyWavelets not installed. Installing...")
    print("Please run: pip install PyWavelets")
    # Create a simple approximation
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.text(0.5, 0.5, 'PyWavelets library required\nfor DWT demonstration', 
            ha='center', va='center', fontsize=14)
    ax.axis('off')
    plt.show()


# 12. Pyramids, Edge Detection, dan Feature Detection/Description

## 1. Gaussian Pyramid

Gaussian Pyramid adalah representasi multi-scale dari citra menggunakan downsampling berulang.


In [ ]:
# Gaussian Pyramid
def build_gaussian_pyramid(img, levels=4):
    pyramid = [img]
    current = img.copy()
    for i in range(levels-1):
        current = cv2.pyrDown(current)
        pyramid.append(current)
    return pyramid

gaussian_pyramid = build_gaussian_pyramid(img_gray, levels=4)

fig, axes = plt.subplots(1, 4, figsize=(16, 4))
for i, level in enumerate(gaussian_pyramid):
    axes[i].imshow(level, cmap='gray')
    axes[i].set_title(f'Level {i} ({level.shape[1]}x{level.shape[0]})')
    axes[i].axis('off')
plt.tight_layout()
plt.show()


## 2. Laplacian Pyramid

Laplacian Pyramid menyimpan perbedaan antara level-level Gaussian Pyramid, berguna untuk kompresi dan blending.


In [ ]:
# Laplacian Pyramid
def build_laplacian_pyramid(img, levels=4):
    gaussian_pyr = build_gaussian_pyramid(img, levels)
    laplacian_pyr = []
    
    for i in range(levels-1):
        size = (gaussian_pyr[i].shape[1], gaussian_pyr[i].shape[0])
        gaussian_expanded = cv2.pyrUp(gaussian_pyr[i+1], dstsize=size)
        laplacian = cv2.subtract(gaussian_pyr[i], gaussian_expanded)
        laplacian_pyr.append(laplacian)
    
    laplacian_pyr.append(gaussian_pyr[-1])  # Top level is the same
    return laplacian_pyr, gaussian_pyr

laplacian_pyramid, gaussian_pyramid = build_laplacian_pyramid(img_gray, levels=4)

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
for i in range(4):
    axes[0,i].imshow(gaussian_pyramid[i], cmap='gray')
    axes[0,i].set_title(f'Gaussian Level {i}')
    axes[0,i].axis('off')
    
    # Normalize Laplacian for display
    lap_display = laplacian_pyramid[i].copy()
    if lap_display.min() < 0:
        lap_display = lap_display - lap_display.min()
    if lap_display.max() > 0:
        lap_display = (lap_display / lap_display.max() * 255).astype(np.uint8)
    
    axes[1,i].imshow(lap_display, cmap='gray')
    axes[1,i].set_title(f'Laplacian Level {i}')
    axes[1,i].axis('off')
plt.tight_layout()
plt.show()


## 3. Canny Edge Detector

Canny edge detector adalah algoritma edge detection yang populer dengan deteksi edge yang baik.


In [ ]:
# Canny Edge Detector
# Different threshold combinations
edges_canny1 = cv2.Canny(img_gray, 50, 150)
edges_canny2 = cv2.Canny(img_gray, 100, 200)
edges_canny3 = cv2.Canny(img_gray, 50, 100)
edges_canny4 = cv2.Canny(img_gray, 150, 250)

# With Gaussian blur preprocessing
img_blurred = cv2.GaussianBlur(img_gray, (5, 5), 1.0)
edges_canny_blur = cv2.Canny(img_blurred, 50, 150)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes[0,0].imshow(img_gray, cmap='gray')
axes[0,0].set_title('Original')
axes[0,0].axis('off')

axes[0,1].imshow(edges_canny1, cmap='gray')
axes[0,1].set_title('Canny (50, 150)')
axes[0,1].axis('off')

axes[0,2].imshow(edges_canny2, cmap='gray')
axes[0,2].set_title('Canny (100, 200)')
axes[0,2].axis('off')

axes[1,0].imshow(edges_canny3, cmap='gray')
axes[1,0].set_title('Canny (50, 100) - More edges')
axes[1,0].axis('off')

axes[1,1].imshow(edges_canny4, cmap='gray')
axes[1,1].set_title('Canny (150, 250) - Fewer edges')
axes[1,1].axis('off')

axes[1,2].imshow(edges_canny_blur, cmap='gray')
axes[1,2].set_title('Canny (with Gaussian blur)')
axes[1,2].axis('off')
plt.tight_layout()
plt.show()


## 4. Feature Detector: Harris Corner Detector dan SIFT

Feature detector mengidentifikasi titik-titik menarik dalam citra (corners, keypoints).


In [ ]:
# Feature Detectors
# Harris Corner Detector
harris = cv2.cornerHarris(img_gray, 2, 3, 0.04)
harris = cv2.dilate(harris, None)
img_harris = img_rgb.copy()
img_harris[harris > 0.01 * harris.max()] = [255, 0, 0]  # Mark corners in red

# Good Features to Track (improved corner detection)
corners = cv2.goodFeaturesToTrack(img_gray, maxCorners=100, qualityLevel=0.01, minDistance=10)
img_corners = img_rgb.copy()
if corners is not None:
    corners = np.int0(corners)
    for i in corners:
        x, y = i.ravel()
        cv2.circle(img_corners, (x, y), 3, (0, 255, 0), -1)

# SIFT Feature Detector
try:
    sift = cv2.SIFT_create()
    keypoints_sift, descriptors_sift = sift.detectAndCompute(img_gray, None)
    img_sift = cv2.drawKeypoints(img_rgb, keypoints_sift, None, 
                                 flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
except:
    img_sift = img_rgb.copy()
    keypoints_sift = []
    print("SIFT not available (requires opencv-contrib-python)")

fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes[0,0].imshow(img_rgb)
axes[0,0].set_title('Original')
axes[0,0].axis('off')

axes[0,1].imshow(img_harris)
axes[0,1].set_title('Harris Corner Detector')
axes[0,1].axis('off')

axes[1,0].imshow(img_corners)
axes[1,0].set_title('Good Features to Track')
axes[1,0].axis('off')

axes[1,1].imshow(img_sift)
axes[1,1].set_title(f'SIFT Keypoints ({len(keypoints_sift)})')
axes[1,1].axis('off')
plt.tight_layout()
plt.show()


## 5. Feature Descriptor: SIFT Feature Descriptor, Shape Context, HOG

Feature descriptor mengkarakterisasi region sekitar keypoint untuk matching dan recognition.


In [ ]:
# Feature Descriptors
# SIFT Descriptor (already computed above, but let's visualize it)
if len(keypoints_sift) > 0:
    print(f"SIFT Descriptors shape: {descriptors_sift.shape}")
    print(f"Number of keypoints: {len(keypoints_sift)}")
    print(f"Descriptor dimension: {descriptors_sift.shape[1]}")
    
    # Visualize descriptor for one keypoint
    if descriptors_sift.shape[0] > 0:
        sample_descriptor = descriptors_sift[0]
        plt.figure(figsize=(12, 3))
        plt.bar(range(len(sample_descriptor)), sample_descriptor)
        plt.title('SIFT Descriptor (First Keypoint)')
        plt.xlabel('Descriptor Index')
        plt.ylabel('Value')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

# HOG (Histogram of Oriented Gradients)
try:
    from skimage.feature import hog
    from skimage import exposure as skexposure
    
    # Compute HOG features
    hog_features, hog_image = hog(img_gray, orientations=9, pixels_per_cell=(8, 8),
                                  cells_per_block=(2, 2), visualize=True, feature_vector=True)
    
    # Rescale histogram for better visualization
    hog_image_rescaled = skexposure.rescale_intensity(hog_image, in_range=(0, 10))
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    axes[0].imshow(img_gray, cmap='gray')
    axes[0].set_title('Original Image')
    axes[0].axis('off')
    
    axes[1].imshow(hog_image_rescaled, cmap='gray')
    axes[1].set_title('HOG Visualization')
    axes[1].axis('off')
    
    axes[2].bar(range(len(hog_features[:100])), hog_features[:100])
    axes[2].set_title('HOG Features (First 100)')
    axes[2].set_xlabel('Feature Index')
    axes[2].set_ylabel('Value')
    axes[2].grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print(f"HOG Features shape: {hog_features.shape}")
    
except ImportError:
    print("scikit-image required for HOG. Using alternative method...")
    # Simple gradient-based descriptor
    sobelx = cv2.Sobel(img_gray, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(img_gray, cv2.CV_64F, 0, 1, ksize=3)
    magnitude = np.sqrt(sobelx**2 + sobely**2)
    angle = np.arctan2(sobely, sobelx)
    
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(magnitude, cmap='gray')
    plt.title('Gradient Magnitude')
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(angle, cmap='hsv')
    plt.title('Gradient Direction')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# ORB Descriptor (alternative to SIFT, faster)
orb = cv2.ORB_create(nfeatures=100)
keypoints_orb, descriptors_orb = orb.detectAndCompute(img_gray, None)
img_orb = cv2.drawKeypoints(img_rgb, keypoints_orb, None, color=(0, 255, 0), flags=0)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].imshow(img_rgb)
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].imshow(img_orb)
axes[1].set_title(f'ORB Keypoints ({len(keypoints_orb)})')
axes[1].axis('off')
plt.tight_layout()
plt.show()

if descriptors_orb is not None:
    print(f"ORB Descriptors shape: {descriptors_orb.shape}")


# 13. Pengolahan Citra Berwarna

## 1. Trichromacy

Trichromacy adalah teori bahwa penglihatan warna manusia didasarkan pada tiga jenis sel kerucut (cones) yang sensitif terhadap panjang gelombang berbeda.


In [ ]:
# Trichromacy - Color Vision
# Simulate cone responses (simplified)
def simulate_cone_response(wavelength):
    """Simulate S, M, L cone responses"""
    # S-cone (short wavelength, blue)
    s_response = np.exp(-((wavelength - 420)**2) / (2 * 30**2))
    # M-cone (medium wavelength, green)
    m_response = np.exp(-((wavelength - 530)**2) / (2 * 30**2))
    # L-cone (long wavelength, red)
    l_response = np.exp(-((wavelength - 560)**2) / (2 * 30**2))
    return s_response, m_response, l_response

wavelengths = np.linspace(400, 700, 300)
s_responses, m_responses, l_responses = [], [], []

for w in wavelengths:
    s, m, l = simulate_cone_response(w)
    s_responses.append(s)
    m_responses.append(m)
    l_responses.append(l)

fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(wavelengths, s_responses, 'b', label='S-cone (Blue)', linewidth=2)
ax.plot(wavelengths, m_responses, 'g', label='M-cone (Green)', linewidth=2)
ax.plot(wavelengths, l_responses, 'r', label='L-cone (Red)', linewidth=2)
ax.set_xlabel('Wavelength (nm)')
ax.set_ylabel('Normalized Response')
ax.set_title('Trichromatic Cone Response Functions')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Demonstrate RGB separation (trichromacy in digital imaging)
img_color = cv2.imread("images/mandrill.jpg", cv2.IMREAD_COLOR)
img_rgb = cv2.cvtColor(img_color, cv2.COLOR_BGR2RGB)

# Split into R, G, B channels
r, g, b = cv2.split(img_rgb)

fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes[0,0].imshow(img_rgb)
axes[0,0].set_title('Original RGB Image')
axes[0,0].axis('off')

axes[0,1].imshow(r, cmap='Reds')
axes[0,1].set_title('Red Channel (L-cone response)')
axes[0,1].axis('off')

axes[1,0].imshow(g, cmap='Greens')
axes[1,0].set_title('Green Channel (M-cone response)')
axes[1,0].axis('off')

axes[1,1].imshow(b, cmap='Blues')
axes[1,1].set_title('Blue Channel (S-cone response)')
axes[1,1].axis('off')
plt.tight_layout()
plt.show()


## 2. Linear Color Transformation

Transformasi warna linier digunakan untuk konversi antar color space dan manipulasi warna.


In [ ]:
# Linear Color Transformation
# RGB to Grayscale (linear combination)
gray_linear = 0.299 * r.astype(float) + 0.587 * g.astype(float) + 0.114 * b.astype(float)
gray_linear = gray_linear.astype(np.uint8)

# Brightness adjustment (linear)
brightness_factor = 1.3
img_bright = np.clip(img_rgb.astype(float) * brightness_factor, 0, 255).astype(np.uint8)

# Contrast adjustment (linear)
contrast_factor = 1.5
img_contrast = np.clip((img_rgb.astype(float) - 128) * contrast_factor + 128, 0, 255).astype(np.uint8)

# Color matrix transformation
# Example: Sepia tone
sepia_matrix = np.array([[0.393, 0.769, 0.189],
                         [0.349, 0.686, 0.168],
                         [0.272, 0.534, 0.131]])
img_sepia = cv2.transform(img_rgb, sepia_matrix)
img_sepia = np.clip(img_sepia, 0, 255).astype(np.uint8)

fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes[0,0].imshow(img_rgb)
axes[0,0].set_title('Original')
axes[0,0].axis('off')

axes[0,1].imshow(img_bright)
axes[0,1].set_title('Brightness Adjusted (×1.3)')
axes[0,1].axis('off')

axes[1,0].imshow(img_contrast)
axes[1,0].set_title('Contrast Adjusted (×1.5)')
axes[1,0].axis('off')

axes[1,1].imshow(img_sepia)
axes[1,1].set_title('Sepia Tone (Linear Transform)')
axes[1,1].axis('off')
plt.tight_layout()
plt.show()


## 3. Color Spaces

Berbagai color space memiliki keunggulan berbeda untuk aplikasi tertentu.


In [ ]:
# Color Spaces
# RGB (already have)
img_rgb = cv2.cvtColor(img_color, cv2.COLOR_BGR2RGB)

# HSV (Hue, Saturation, Value)
img_hsv = cv2.cvtColor(img_color, cv2.COLOR_BGR2HSV)
h, s, v = cv2.split(img_hsv)

# LAB (L*a*b*)
img_lab = cv2.cvtColor(img_color, cv2.COLOR_BGR2LAB)
l, a, b = cv2.split(img_lab)

# YUV / YCbCr
img_yuv = cv2.cvtColor(img_color, cv2.COLOR_BGR2YUV)
y, u, v_comp = cv2.split(img_yuv)

# XYZ
img_xyz = cv2.cvtColor(img_color, cv2.COLOR_BGR2XYZ)
x, y_comp, z = cv2.split(img_xyz)

# Visualize different color spaces
fig, axes = plt.subplots(4, 3, figsize=(15, 20))

# RGB
axes[0,0].imshow(img_rgb)
axes[0,0].set_title('RGB')
axes[0,0].axis('off')

axes[0,1].imshow(r, cmap='Reds')
axes[0,1].set_title('RGB - Red')
axes[0,1].axis('off')

axes[0,2].imshow(g, cmap='Greens')
axes[0,2].set_title('RGB - Green')
axes[0,2].axis('off')

# HSV
axes[1,0].imshow(cv2.cvtColor(img_hsv, cv2.COLOR_HSV2RGB))
axes[1,0].set_title('HSV (converted to RGB for display)')
axes[1,0].axis('off')

axes[1,1].imshow(h, cmap='hsv')
axes[1,1].set_title('HSV - Hue')
axes[1,1].axis('off')

axes[1,2].imshow(s, cmap='gray')
axes[1,2].set_title('HSV - Saturation')
axes[1,2].axis('off')

# LAB
axes[2,0].imshow(cv2.cvtColor(img_lab, cv2.COLOR_LAB2RGB))
axes[2,0].set_title('LAB (converted to RGB for display)')
axes[2,0].axis('off')

axes[2,1].imshow(l, cmap='gray')
axes[2,1].set_title('LAB - L* (Lightness)')
axes[2,1].axis('off')

axes[2,2].imshow(a, cmap='RdYlGn')
axes[2,2].set_title('LAB - a* (Green-Red)')
axes[2,2].axis('off')

# YUV
axes[3,0].imshow(cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB))
axes[3,0].set_title('YUV (converted to RGB for display)')
axes[3,0].axis('off')

axes[3,1].imshow(y, cmap='gray')
axes[3,1].set_title('YUV - Y (Luminance)')
axes[3,1].axis('off')

axes[3,2].imshow(u, cmap='gray')
axes[3,2].set_title('YUV - U (Chrominance)')
axes[3,2].axis('off')

plt.tight_layout()
plt.show()

# Color space comparison for specific tasks
# Example: Skin detection in HSV
lower_skin = np.array([0, 20, 70], dtype=np.uint8)
upper_skin = np.array([20, 255, 255], dtype=np.uint8)
skin_mask = cv2.inRange(img_hsv, lower_skin, upper_skin)
skin_detected = cv2.bitwise_and(img_rgb, img_rgb, mask=skin_mask)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(img_rgb)
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].imshow(skin_mask, cmap='gray')
axes[1].set_title('Skin Detection Mask (HSV)')
axes[1].axis('off')

axes[2].imshow(skin_detected)
axes[2].set_title('Detected Skin')
axes[2].axis('off')
plt.tight_layout()
plt.show()


# 14. Berbagai Metode Segmentasi pada Citra

Segmentasi adalah proses membagi citra menjadi region-region yang bermakna.


In [ ]:
# Image Segmentation Methods
# 1. Threshold-based Segmentation
_, thresh_seg = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# 2. Region Growing (using watershed as approximation)
# Create markers
_, markers = cv2.connectedComponents(thresh_seg)
markers = markers + 1
markers[thresh_seg == 0] = 0

# Apply watershed
img_for_watershed = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
cv2.watershed(img_for_watershed, markers)
watershed_result = markers.astype(np.uint8)

# 3. K-means Clustering Segmentation
data = img_rgb.reshape((-1, 3))
data = np.float32(data)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 1.0)
k = 4
_, labels, centers = cv2.kmeans(data, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
centers = np.uint8(centers)
segmented_kmeans = centers[labels.flatten()]
segmented_kmeans = segmented_kmeans.reshape(img_rgb.shape)

# 4. Mean Shift Segmentation
try:
    from sklearn.cluster import MeanShift
    data_reshaped = img_rgb.reshape((-1, 3))
    ms = MeanShift(bandwidth=20, bin_seeding=True)
    ms.fit(data_reshaped[:10000])  # Sample for speed
    labels_ms = ms.predict(data_reshaped)
    segmented_meanshift = ms.cluster_centers_[labels_ms].astype(np.uint8)
    segmented_meanshift = segmented_meanshift.reshape(img_rgb.shape)
except ImportError:
    segmented_meanshift = img_rgb.copy()
    print("sklearn required for Mean Shift")

# 5. GrabCut Segmentation
mask = np.zeros(img_gray.shape[:2], np.uint8)
bgdModel = np.zeros((1, 65), np.float64)
fgdModel = np.zeros((1, 65), np.float64)
rect = (50, 50, img_gray.shape[1]-100, img_gray.shape[0]-100)
cv2.grabCut(img_rgb, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)
mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
grabcut_result = img_rgb * mask2[:, :, np.newaxis]

# 6. Contour-based Segmentation
contours, _ = cv2.findContours(thresh_seg, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
img_contours = img_rgb.copy()
cv2.drawContours(img_contours, contours, -1, (0, 255, 0), 2)

fig, axes = plt.subplots(3, 2, figsize=(15, 20))
axes[0,0].imshow(img_rgb)
axes[0,0].set_title('Original')
axes[0,0].axis('off')

axes[0,1].imshow(thresh_seg, cmap='gray')
axes[0,1].set_title('Threshold Segmentation')
axes[0,1].axis('off')

axes[1,0].imshow(segmented_kmeans)
axes[1,0].set_title('K-means Clustering (k=4)')
axes[1,0].axis('off')

axes[1,1].imshow(segmented_meanshift)
axes[1,1].set_title('Mean Shift Segmentation')
axes[1,1].axis('off')

axes[2,0].imshow(grabcut_result)
axes[2,0].set_title('GrabCut Segmentation')
axes[2,0].axis('off')

axes[2,1].imshow(img_contours)
axes[2,1].set_title('Contour-based Segmentation')
axes[2,1].axis('off')
plt.tight_layout()
plt.show()

# Advanced: SLIC Superpixels
try:
    from skimage.segmentation import slic, mark_boundaries
    segments_slic = slic(img_rgb, n_segments=100, compactness=10, sigma=1)
    img_slic = mark_boundaries(img_rgb, segments_slic)
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 7))
    axes[0].imshow(img_rgb)
    axes[0].set_title('Original')
    axes[0].axis('off')
    
    axes[1].imshow(img_slic)
    axes[1].set_title('SLIC Superpixels (100 segments)')
    axes[1].axis('off')
    plt.tight_layout()
    plt.show()
except ImportError:
    print("scikit-image required for SLIC superpixels")


In [ ]:
# Image Compression Concepts
import os

# Original image size
original_size = os.path.getsize("images/mandrill.jpg") if os.path.exists("images/mandrill.jpg") else 0
print(f"Original JPEG size: {original_size / 1024:.2f} KB")

# Save as different formats to compare
cv2.imwrite("temp_original.png", img_rgb)
png_size = os.path.getsize("temp_original.png") if os.path.exists("temp_original.png") else 0
print(f"PNG (lossless) size: {png_size / 1024:.2f} KB")

# JPEG with different quality (lossy)
cv2.imwrite("temp_jpeg_high.jpg", img_rgb, [cv2.IMWRITE_JPEG_QUALITY, 95])
cv2.imwrite("temp_jpeg_medium.jpg", img_rgb, [cv2.IMWRITE_JPEG_QUALITY, 50])
cv2.imwrite("temp_jpeg_low.jpg", img_rgb, [cv2.IMWRITE_JPEG_QUALITY, 10])

jpeg_high = os.path.getsize("temp_jpeg_high.jpg") if os.path.exists("temp_jpeg_high.jpg") else 0
jpeg_medium = os.path.getsize("temp_jpeg_medium.jpg") if os.path.exists("temp_jpeg_medium.jpg") else 0
jpeg_low = os.path.getsize("temp_jpeg_low.jpg") if os.path.exists("temp_jpeg_low.jpg") else 0

print(f"JPEG Quality 95: {jpeg_high / 1024:.2f} KB")
print(f"JPEG Quality 50: {jpeg_medium / 1024:.2f} KB")
print(f"JPEG Quality 10: {jpeg_low / 1024:.2f} KB")

# Load and compare
img_jpeg_high = cv2.imread("temp_jpeg_high.jpg")
img_jpeg_high = cv2.cvtColor(img_jpeg_high, cv2.COLOR_BGR2RGB)
img_jpeg_medium = cv2.imread("temp_jpeg_medium.jpg")
img_jpeg_medium = cv2.cvtColor(img_jpeg_medium, cv2.COLOR_BGR2RGB)
img_jpeg_low = cv2.imread("temp_jpeg_low.jpg")
img_jpeg_low = cv2.cvtColor(img_jpeg_low, cv2.COLOR_BGR2RGB)

fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes[0,0].imshow(img_rgb)
axes[0,0].set_title(f'Original PNG\n({png_size/1024:.1f} KB)')
axes[0,0].axis('off')

axes[0,1].imshow(img_jpeg_high)
axes[0,1].set_title(f'JPEG Quality 95\n({jpeg_high/1024:.1f} KB)')
axes[0,1].axis('off')

axes[1,0].imshow(img_jpeg_medium)
axes[1,0].set_title(f'JPEG Quality 50\n({jpeg_medium/1024:.1f} KB)')
axes[1,0].axis('off')

axes[1,1].imshow(img_jpeg_low)
axes[1,1].set_title(f'JPEG Quality 10\n({jpeg_low/1024:.1f} KB)')
axes[1,1].axis('off')
plt.tight_layout()
plt.show()

# Calculate PSNR (Peak Signal-to-Noise Ratio)
def calculate_psnr(img1, img2):
    mse = np.mean((img1.astype(float) - img2.astype(float)) ** 2)
    if mse == 0:
        return float('inf')
    return 20 * np.log10(255.0 / np.sqrt(mse))

psnr_high = calculate_psnr(img_rgb, img_jpeg_high)
psnr_medium = calculate_psnr(img_rgb, img_jpeg_medium)
psnr_low = calculate_psnr(img_rgb, img_jpeg_low)

print(f"\nPSNR (Quality 95): {psnr_high:.2f} dB")
print(f"PSNR (Quality 50): {psnr_medium:.2f} dB")
print(f"PSNR (Quality 10): {psnr_low:.2f} dB")


## 2. Lossless Compression: Huffman Coding, RLE, Arithmetic Coding

Lossless compression mempertahankan semua informasi citra.


In [ ]:
# Lossless Compression Methods
# Run-Length Encoding (RLE)
def rle_encode(image):
    """Simple RLE encoding for demonstration"""
    flat = image.flatten()
    encoded = []
    count = 1
    for i in range(1, len(flat)):
        if flat[i] == flat[i-1]:
            count += 1
        else:
            encoded.append((flat[i-1], count))
            count = 1
    encoded.append((flat[-1], count))
    return encoded

def rle_decode(encoded, shape):
    """RLE decoding"""
    decoded = []
    for value, count in encoded:
        decoded.extend([value] * count)
    return np.array(decoded).reshape(shape)

# Apply RLE to binary image
binary_for_rle = (img_gray > 127).astype(np.uint8) * 255
rle_encoded = rle_encode(binary_for_rle)
rle_decoded = rle_decode(rle_encoded, binary_for_rle.shape)

original_size_bits = binary_for_rle.size * 8
rle_size_bits = len(rle_encoded) * (8 + 16)  # Approximate: value (8 bits) + count (16 bits)
compression_ratio_rle = original_size_bits / rle_size_bits if rle_size_bits > 0 else 1

print(f"Original size: {original_size_bits} bits")
print(f"RLE encoded size: {rle_size_bits} bits")
print(f"Compression ratio: {compression_ratio_rle:.2f}:1")

# Verify lossless
assert np.array_equal(binary_for_rle, rle_decoded), "RLE should be lossless!"

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].imshow(binary_for_rle, cmap='gray')
axes[0].set_title('Original Binary Image')
axes[0].axis('off')

axes[1].imshow(rle_decoded, cmap='gray')
axes[1].set_title('RLE Decoded (Lossless)')
axes[1].axis('off')
plt.tight_layout()
plt.show()

# Huffman Coding (simplified demonstration)
from collections import Counter

def simple_huffman_demo(image):
    """Demonstrate Huffman coding concept"""
    # Count frequency of each pixel value
    pixel_counts = Counter(image.flatten())
    total_pixels = image.size
    
    # Calculate entropy (theoretical minimum bits per pixel)
    entropy = -sum((count/total_pixels) * np.log2(count/total_pixels) 
                   for count in pixel_counts.values() if count > 0)
    
    return entropy, pixel_counts

entropy, pixel_counts = simple_huffman_demo(img_gray)
print(f"\nImage Entropy: {entropy:.2f} bits/pixel")
print(f"Original: 8 bits/pixel")
print(f"Theoretical compression: {8/entropy:.2f}:1 (if entropy < 8)")

# Note: Full Huffman implementation requires tree building and encoding
# This is a simplified demonstration of the concept


## 3. Lossy Compression: DFT, WHT, DCT, KLT

Lossy compression menggunakan transformasi untuk mengurangi redundansi dengan mengorbankan beberapa informasi.


In [ ]:
# Lossy Compression: Transform-based Methods
# 1. DCT (Discrete Cosine Transform) - used in JPEG
from scipy.fftpack import dct, idct

def dct_compress(image, block_size=8, keep_ratio=0.1):
    """DCT-based compression"""
    h, w = image.shape
    compressed = np.zeros_like(image, dtype=float)
    
    for i in range(0, h, block_size):
        for j in range(0, w, block_size):
            block = image[i:i+block_size, j:j+block_size].astype(float) - 128
            dct_block = dct(dct(block, axis=0, norm='ortho'), axis=1, norm='ortho')
            
            # Keep only top coefficients
            threshold = np.sort(np.abs(dct_block.flatten()))[-int(block_size*block_size*keep_ratio)]
            dct_block[np.abs(dct_block) < threshold] = 0
            
            # Inverse DCT
            idct_block = idct(idct(dct_block, axis=0, norm='ortho'), axis=1, norm='ortho')
            compressed[i:i+block_size, j:j+block_size] = idct_block + 128
    
    return np.clip(compressed, 0, 255).astype(np.uint8)

# Apply DCT compression with different ratios
img_dct_10 = dct_compress(img_gray, keep_ratio=0.1)
img_dct_5 = dct_compress(img_gray, keep_ratio=0.05)
img_dct_1 = dct_compress(img_gray, keep_ratio=0.01)

# 2. DFT Compression (already demonstrated, but for compression)
f_transform = fft2(img_gray)
f_shift = fftshift(f_transform)
magnitude = np.abs(f_shift)

# Keep only top frequencies
keep_ratio_dft = 0.1
threshold_dft = np.sort(magnitude.flatten())[-int(magnitude.size * keep_ratio_dft)]
mask_dft = magnitude >= threshold_dft
f_compressed = f_shift * mask_dft
img_dft_compressed = np.abs(ifft2(ifftshift(f_compressed))).astype(np.uint8)

# 3. Walsh-Hadamard Transform (WHT) - simplified
def hadamard_matrix(n):
    """Generate Hadamard matrix"""
    if n == 1:
        return np.array([[1]])
    h = hadamard_matrix(n // 2)
    return np.block([[h, h], [h, -h]])

# Note: Full WHT implementation is complex, showing concept
try:
    # Use scipy's hadamard if available
    from scipy.linalg import hadamard
    n = 8
    H = hadamard(n)
    # Apply to 8x8 blocks (simplified)
    print("WHT: Using Hadamard transform for compression concept")
except:
    print("WHT: Concept demonstrated (full implementation requires larger matrices)")

# Visualize compression results
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes[0,0].imshow(img_gray, cmap='gray')
axes[0,0].set_title('Original')
axes[0,0].axis('off')

axes[0,1].imshow(img_dct_10, cmap='gray')
axes[0,1].set_title('DCT (10% coefficients)')
axes[0,1].axis('off')

axes[0,2].imshow(img_dct_5, cmap='gray')
axes[0,2].set_title('DCT (5% coefficients)')
axes[0,2].axis('off')

axes[1,0].imshow(img_dct_1, cmap='gray')
axes[1,0].set_title('DCT (1% coefficients)')
axes[1,0].axis('off')

axes[1,1].imshow(img_dft_compressed, cmap='gray')
axes[1,1].set_title('DFT (10% frequencies)')
axes[1,1].axis('off')

# Calculate compression metrics
psnr_dct_10 = calculate_psnr(img_gray, img_dct_10)
psnr_dct_5 = calculate_psnr(img_gray, img_dct_5)
psnr_dct_1 = calculate_psnr(img_gray, img_dct_1)
psnr_dft = calculate_psnr(img_gray, img_dft_compressed)

axes[1,2].axis('off')
plt.tight_layout()
plt.show()

print(f"\nCompression Quality Metrics:")
print(f"DCT (10%): PSNR = {psnr_dct_10:.2f} dB")
print(f"DCT (5%): PSNR = {psnr_dct_5:.2f} dB")
print(f"DCT (1%): PSNR = {psnr_dct_1:.2f} dB")
print(f"DFT (10%): PSNR = {psnr_dft:.2f} dB")

# KLT (Karhunen-Loeve Transform / PCA) - Principal Component Analysis
from sklearn.decomposition import PCA

# Reshape image for PCA
h, w = img_gray.shape
img_flat = img_gray.reshape(-1, 1)
pca = PCA(n_components=1)
pca.fit(img_flat)
img_pca = pca.inverse_transform(pca.transform(img_flat))
img_pca = img_pca.reshape(h, w).astype(np.uint8)

# For color image PCA
img_rgb_flat = img_rgb.reshape(-1, 3)
pca_rgb = PCA(n_components=50)  # Keep 50 principal components
img_rgb_pca = pca_rgb.inverse_transform(pca_rgb.transform(img_rgb_flat))
img_rgb_pca = np.clip(img_rgb_pca, 0, 255).astype(np.uint8)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].imshow(img_rgb, cmap='gray' if len(img_rgb.shape) == 2 else None)
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].imshow(img_rgb_pca, cmap='gray' if len(img_rgb_pca.shape) == 2 else None)
axes[1].set_title(f'PCA Compression (50 components, {pca_rgb.explained_variance_ratio_.sum()*100:.1f}% variance)')
axes[1].axis('off')
plt.tight_layout()
plt.show()


In [ ]:
# Applications of Digital Image Processing

# 1. Medical Imaging Applications
print("=== MEDICAL IMAGING APPLICATIONS ===\n")

print("1. X-Ray Image Enhancement:")
print("   - Contrast enhancement untuk meningkatkan visibility")
print("   - Noise reduction untuk clarity")
print("   - Edge detection untuk identifikasi fractures")

print("\n2. CT Scan Analysis:")
print("   - 3D reconstruction dari 2D slices")
print("   - Tumor detection dan segmentation")
print("   - Volume measurement")

print("\n3. MRI Processing:")
print("   - Image registration untuk alignment")
print("   - Brain segmentation")
print("   - Functional MRI analysis")

print("\n4. Ultrasound Imaging:")
print("   - Speckle noise reduction")
print("   - Fetal development monitoring")
print("   - Organ boundary detection")

# Simulate medical image processing
# Create a synthetic medical-like image
medical_synthetic = np.random.normal(128, 30, img_gray.shape).astype(np.uint8)
medical_synthetic = cv2.GaussianBlur(medical_synthetic, (5, 5), 2)

# Apply medical imaging techniques
# Contrast enhancement
medical_enhanced = cv2.equalizeHist(medical_synthetic)

# Edge detection for structure identification
medical_edges = cv2.Canny(medical_synthetic, 50, 150)

# Segmentation
_, medical_thresh = cv2.threshold(medical_synthetic, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes[0,0].imshow(medical_synthetic, cmap='gray')
axes[0,0].set_title('Synthetic Medical Image')
axes[0,0].axis('off')

axes[0,1].imshow(medical_enhanced, cmap='gray')
axes[0,1].set_title('Enhanced (Histogram Equalization)')
axes[0,1].axis('off')

axes[1,0].imshow(medical_edges, cmap='gray')
axes[1,0].set_title('Edge Detection (Structure Identification)')
axes[1,0].axis('off')

axes[1,1].imshow(medical_thresh, cmap='gray')
axes[1,1].set_title('Segmentation (Threshold)')
axes[1,1].axis('off')
plt.tight_layout()
plt.show()

# 2. Scientific Applications
print("\n=== SCIENTIFIC APPLICATIONS ===\n")

print("1. Astronomy:")
print("   - Star detection dan classification")
print("   - Galaxy morphology analysis")
print("   - Image stacking untuk noise reduction")

print("\n2. Remote Sensing:")
print("   - Land use classification")
print("   - Vegetation index calculation")
print("   - Change detection")

print("\n3. Microscopy:")
print("   - Cell counting dan analysis")
print("   - Particle size measurement")
print("   - Fluorescence imaging")

print("\n4. Material Science:")
print("   - Grain size analysis")
print("   - Defect detection")
print("   - Surface texture analysis")

# Demonstrate remote sensing concept
# Simulate multi-spectral image
bands = []
for i in range(3):
    band = np.random.normal(100 + i*30, 20, img_gray.shape).astype(np.uint8)
    bands.append(band)

multispectral = np.stack(bands, axis=2)
ndvi_like = ((bands[2].astype(float) - bands[0].astype(float)) / 
             (bands[2].astype(float) + bands[0].astype(float) + 1))
ndvi_like = ((ndvi_like + 1) * 127.5).astype(np.uint8)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(multispectral)
axes[0].set_title('Multi-spectral Image (Simulated)')
axes[0].axis('off')

axes[1].imshow(ndvi_like, cmap='RdYlGn')
axes[1].set_title('Vegetation Index (NDVI-like)')
axes[1].axis('off')

axes[2].imshow(cv2.cvtColor(multispectral, cv2.COLOR_BGR2RGB))
axes[2].set_title('Color Composite')
axes[2].axis('off')
plt.tight_layout()
plt.show()

# 3. Industrial Applications
print("\n=== INDUSTRIAL APPLICATIONS ===\n")

print("1. Quality Control:")
print("   - Defect detection")
print("   - Dimensional measurement")
print("   - Surface inspection")

print("\n2. Robotics:")
print("   - Object recognition")
print("   - Path planning")
print("   - Visual servoing")

print("\n3. Security:")
print("   - Face recognition")
print("   - Motion detection")
print("   - License plate recognition")

# Demonstrate quality control concept
# Create synthetic product image with defect
product = np.ones((200, 200), dtype=np.uint8) * 200
product[50:150, 50:150] = 180  # Main object
product[90:110, 90:110] = 50   # Defect (darker region)

# Detect defect
_, product_binary = cv2.threshold(product, 150, 255, cv2.THRESH_BINARY)
contours, _ = cv2.findContours(product_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
product_marked = cv2.cvtColor(product, cv2.COLOR_GRAY2RGB)
cv2.drawContours(product_marked, contours, -1, (255, 0, 0), 2)

# Mark defect
defect_mask = product < 100
product_marked[defect_mask] = [255, 0, 0]

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(product, cmap='gray')
axes[0].set_title('Product Image')
axes[0].axis('off')

axes[1].imshow(product_binary, cmap='gray')
axes[1].set_title('Segmentation')
axes[1].axis('off')

axes[2].imshow(product_marked)
axes[2].set_title('Defect Detection')
axes[2].axis('off')
plt.tight_layout()
plt.show()


In [ ]:
# State of the Art in Digital Image Processing

print("=== STATE OF THE ART TECHNIQUES ===\n")

print("1. DEEP LEARNING FOR IMAGE PROCESSING:")
print("   - Convolutional Neural Networks (CNN) untuk classification")
print("   - U-Net untuk image segmentation")
print("   - GANs (Generative Adversarial Networks) untuk image generation")
print("   - Autoencoders untuk denoising dan compression")
print("   - Transfer learning untuk domain adaptation")

print("\n2. COMPUTER VISION ADVANCES:")
print("   - Object detection: YOLO, R-CNN, SSD")
print("   - Semantic segmentation: DeepLab, FCN")
print("   - Instance segmentation: Mask R-CNN")
print("   - Pose estimation: OpenPose, MediaPipe")

print("\n3. IMAGE ENHANCEMENT:")
print("   - Super-resolution: SRCNN, ESRGAN")
print("   - Denoising: DnCNN, Noise2Noise")
print("   - Style transfer: Neural Style Transfer")
print("   - Colorization: Deep colorization networks")

print("\n4. MEDICAL IMAGING AI:")
print("   - Automated diagnosis: CNN untuk pathology detection")
print("   - Image registration: Deep learning-based registration")
print("   - 3D reconstruction: Deep learning dari 2D slices")
print("   - Anomaly detection: Autoencoders untuk anomaly detection")

# Demonstrate concept of modern techniques
# 1. Super-resolution concept (using interpolation as approximation)
img_small = cv2.resize(img_gray, (img_gray.shape[1]//4, img_gray.shape[0]//4))
img_sr_bicubic = cv2.resize(img_small, (img_gray.shape[1], img_gray.shape[0]), 
                            interpolation=cv2.INTER_CUBIC)
img_sr_lanczos = cv2.resize(img_small, (img_gray.shape[1], img_gray.shape[0]), 
                            interpolation=cv2.INTER_LANCZOS4)

# 2. Denoising concept (using advanced filters)
img_noisy_art = img_gray.copy()
noise_art = np.random.normal(0, 30, img_gray.shape).astype(np.int16)
img_noisy_art = np.clip(img_gray.astype(np.int16) + noise_art, 0, 255).astype(np.uint8)

# Advanced denoising
img_denoised_nlm = cv2.fastNlMeansDenoising(img_noisy_art, None, 10, 7, 21)
img_denoised_bilateral = cv2.bilateralFilter(img_noisy_art, 9, 75, 75)

# 3. Style transfer concept (simplified - using color mapping)
img_style = cv2.imread("images/mandrill.jpg", cv2.IMREAD_COLOR)
if img_style is not None:
    img_style = cv2.cvtColor(img_style, cv2.COLOR_BGR2RGB)
    # Apply color transfer (simplified style transfer)
    img_style_gray = cv2.cvtColor(img_style, cv2.COLOR_RGB2GRAY)
    img_style_transfer = exposure.match_histograms(img_rgb, img_style, channel_axis=2)
else:
    img_style_transfer = img_rgb.copy()

fig, axes = plt.subplots(3, 3, figsize=(15, 15))

# Super-resolution
axes[0,0].imshow(img_small, cmap='gray')
axes[0,0].set_title('Low Resolution (1/4 size)')
axes[0,0].axis('off')

axes[0,1].imshow(img_sr_bicubic, cmap='gray')
axes[0,1].set_title('Super-Resolution (Bicubic)')
axes[0,1].axis('off')

axes[0,2].imshow(img_gray, cmap='gray')
axes[0,2].set_title('Original (Reference)')
axes[0,2].axis('off')

# Denoising
axes[1,0].imshow(img_noisy_art, cmap='gray')
axes[1,0].set_title('Noisy Image')
axes[1,0].axis('off')

axes[1,1].imshow(img_denoised_nlm, cmap='gray')
axes[1,1].set_title('Denoised (Non-local Means)')
axes[1,1].axis('off')

axes[1,2].imshow(img_denoised_bilateral, cmap='gray')
axes[1,2].set_title('Denoised (Bilateral)')
axes[1,2].axis('off')

# Style transfer
axes[2,0].imshow(img_rgb)
axes[2,0].set_title('Original')
axes[2,0].axis('off')

axes[2,1].imshow(img_style_transfer)
axes[2,1].set_title('Style Transfer (Color)')
axes[2,1].axis('off')

axes[2,2].axis('off')
plt.tight_layout()
plt.show()

# 4. Modern feature detection (already shown SIFT, ORB)
# Show modern alternatives
print("\n=== MODERN FEATURE DETECTION ===")
print("Traditional: SIFT, SURF, ORB")
print("Modern: Learned features dari deep networks")
print("   - SuperPoint: Learned feature detector")
print("   - D2-Net: Dense feature detection")
print("   - R2D2: Repeatable and Reliable Detector")

# 5. Segmentation advances
print("\n=== MODERN SEGMENTATION ===")
print("Traditional: Threshold, Region growing, Watershed")
print("Modern Deep Learning:")
print("   - U-Net: Medical image segmentation")
print("   - DeepLab: Semantic segmentation")
print("   - Mask R-CNN: Instance segmentation")
print("   - Segment Anything Model (SAM): Foundation model")

# Summary visualization
techniques_summary = {
    'Traditional': ['Thresholding', 'Edge Detection', 'Morphology', 'Filtering'],
    'Classical ML': ['K-means', 'Mean Shift', 'SVM', 'Random Forest'],
    'Deep Learning': ['CNN', 'U-Net', 'GANs', 'Transformers']
}

fig, ax = plt.subplots(figsize=(10, 6))
y_pos = np.arange(len(techniques_summary))
categories = list(techniques_summary.keys())
counts = [len(techniques_summary[k]) for k in categories]

bars = ax.barh(categories, counts, color=['#3498db', '#2ecc71', '#e74c3c'])
ax.set_xlabel('Number of Techniques')
ax.set_title('Evolution of Image Processing Techniques')
ax.set_xlim(0, max(counts) + 1)

for i, (category, count) in enumerate(zip(categories, counts)):
    ax.text(count + 0.1, i, str(count), va='center', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n=== FUTURE DIRECTIONS ===")
print("1. Foundation Models: Large pre-trained models (e.g., SAM, CLIP)")
print("2. Few-shot Learning: Learning from limited data")
print("3. Explainable AI: Understanding model decisions")
print("4. Edge AI: Processing on mobile/embedded devices")
print("5. Multimodal Learning: Combining vision with other modalities")
print("6. Real-time Processing: Optimized algorithms for speed")
print("7. Privacy-preserving: Federated learning, differential privacy")


## Kesimpulan

Notebook ini telah mencakup materi lengkap Pengolahan Citra Digital (PCD) mulai dari konsep dasar hingga aplikasi modern. Topik yang dibahas meliputi:

1. **Fundamental Concepts**: Definisi, level-level operasi, persepsi visual
2. **Basic Operations**: Operasi geometrik, transformasi intensitas, histogram
3. **Advanced Processing**: Morfologi, filtering, transformasi domain frekuensi
4. **Feature Analysis**: Edge detection, feature detection dan description
5. **Color Processing**: Trichromacy, transformasi warna, color spaces
6. **Segmentation**: Berbagai metode segmentasi citra
7. **Compression**: Lossless dan lossy compression techniques
8. **Applications**: Aplikasi di berbagai bidang sains dan medical imaging
9. **State of the Art**: Teknologi terdepan menggunakan deep learning

Semua konsep dilengkapi dengan implementasi praktis menggunakan OpenCV, scikit-image, dan library Python lainnya.


In [ ]:
# Cleanup temporary files
import os
temp_files = ["temp_original.png", "temp_jpeg_high.jpg", "temp_jpeg_medium.jpg", "temp_jpeg_low.jpg"]
for file in temp_files:
    if os.path.exists(file):
        try:
            os.remove(file)
            print(f"Removed {file}")
        except:
            pass

print("\n=== NOTEBOOK SELESAI ===")
print("Semua materi PCD telah tercakup dalam notebook ini.")
print("Silakan jalankan setiap cell secara berurutan untuk melihat hasilnya.")
